<a href="https://colab.research.google.com/github/alayuala/114-1PL.repo/blob/main/hw3_week6_%E5%BE%85%E8%BE%A6%E6%B8%85%E5%96%AE%E8%88%87%E7%95%AA%E8%8C%84%E9%90%98%E7%B4%80%E9%8C%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **待辦清單與番茄鐘紀錄（作業三）**  
* 目標：在 Sheet 維護 To-do（狀態、預估時間、完成時間）→ Colab 輸出今日計畫與完成率 → 寫回完成標記。  
* AI 點子：讓模型把今日任務排成三段行動計畫（morning/afternoon/evening）。


In [112]:
import os, time, uuid, re, json, datetime
from datetime import datetime as dt, timedelta
from dateutil.tz import gettz
import pandas as pd
import gradio as gr
import requests
from bs4 import BeautifulSoup

import google.generativeai as genai

from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.auth import default
from google.colab import userdata
import io
from urllib.parse import urljoin

# Google Auth & Sheets Setup
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Configuration
# 請將這裡的 SHEET_URL 替換成您自己的 Google Sheet URL (手動輸入)
SHEET_URL = "https://docs.google.com/spreadsheets/d/1ifyMD92D0i7Cq8f8NPJYLt3zafNIQn5SZuvR2OFh0EQ/edit?usp=sharing" # <<<< 請在此處手動輸入您的 Google Sheet URL >>>>
WORKSHEET_NAME = "ProductivityApp"
TIMEZONE = "Asia/Taipei"

# Define Headers
TASKS_HEADER = [
    "id","task","status","priority","est_min","start_time","end_time",
    "actual_min","pomodoros","due_date","labels","notes",
    "created_at","updated_at","completed_at","planned_for"
]
LOGS_HEADER = [
    "log_id","task_id","phase","start_ts","end_ts","minutes","cycles","note"
]
CLIPS_HEADER = ["clip_id","url","selector","text","href","created_at","added_to_task"]

# Ensure Spreadsheet and Worksheets
def ensure_spreadsheet(name):
    try:
        sh = gc.open_by_url(SHEET_URL) # Use open_by_url
    except gspread.SpreadsheetNotFound:
        print(f"Spreadsheet not found at URL: {SHEET_URL}. Please check the URL or ensure the sheet exists.")
        return None
    except Exception as e:
        print(f"Error opening spreadsheet: {e}")
        return None
    return sh


def ensure_worksheet(sh, title, header):
    if sh is None:
        return None
    try:
        ws = sh.worksheet(title)
    except gspread.WorksheetNotFound:
        ws = sh.add_worksheet(title=title, rows="1000", cols=str(len(header)+5))
        ws.update([header])
    data = ws.get_all_values()
    if not data or (data and data[0] != header):
        ws.clear()
        ws.update([header])
    return ws

# --- Execute after SHEET_URL is manually set ---
main_sh = ensure_spreadsheet(WORKSHEET_NAME)
if main_sh:
    ws_tasks = ensure_worksheet(main_sh, "tasks", TASKS_HEADER)
    ws_logs  = ensure_worksheet(main_sh, "pomodoro_logs", LOGS_HEADER)
    ws_clips = ensure_worksheet(main_sh, "web_clips", CLIPS_HEADER)
    print(f"Successfully connected to spreadsheet: {WORKSHEET_NAME}")
    print("Worksheets 'tasks', 'pomodoro_logs', and 'web_clips' are ready.")
else:
    print("Could not connect to spreadsheet. Please check the SHEET_URL.")

# Helper Functions (need to be defined before loading data)
def tznow():
    return dt.now(gettz(TIMEZONE))

def read_df(ws, header):
    if ws is None:
        return pd.DataFrame(columns=header)
    df = get_as_dataframe(ws, evaluate_formulas=True, header=0)
    if df is None or df.empty:
        return pd.DataFrame(columns=header)
    df = df.fillna("")
    for c in header:
        if c not in df.columns:
            df[c] = ""
    if "est_min" in df.columns:
        df["est_min"] = pd.to_numeric(df["est_min"], errors="coerce").fillna(0).astype(int)
    if "actual_min" in df.columns:
        df["actual_min"] = pd.to_numeric(df["actual_min"], errors="coerce").fillna(0).astype(int)
    if "pomodoros" in df.columns:
        df["pomodoros"] = pd.to_numeric(df["pomodoros"], errors="coerce").fillna(0).astype(int)
    return df[header]

def write_df(ws, df, header):
    if ws is None:
        print("Warning: Cannot write to Sheet. Worksheet not initialized.")
        return
    if df.empty:
        ws.clear()
        ws.update([header])
        return
    df_out = df.copy()
    for c in df_out.columns:
        df_out[c] = df_out[c].astype(str)
    ws.clear()
    ws.update([header] + df_out.values.tolist())

def refresh_all():
    # Ensure worksheets are initialized before reading
    if 'ws_tasks' not in globals() or ws_tasks is None:
         main_sh = ensure_spreadsheet(WORKSHEET_NAME)
         if main_sh:
             globals()['ws_tasks'] = ensure_worksheet(main_sh, "tasks", TASKS_HEADER)
             globals()['ws_logs'] = ensure_worksheet(main_sh, "pomodoro_logs", LOGS_HEADER)
             globals()['ws_clips'] = ensure_worksheet(main_sh, "web_clips", CLIPS_HEADER)
         else:
             print("Could not initialize worksheets during refresh.")
             return pd.DataFrame(columns=TASKS_HEADER), pd.DataFrame(columns=LOGS_HEADER), pd.DataFrame(columns=CLIPS_HEADER)


    return (
        read_df(ws_tasks, TASKS_HEADER).copy(),
        read_df(ws_logs, LOGS_HEADER).copy(),
        read_df(ws_clips, CLIPS_HEADER).copy()
    )


# Load data initially (will be empty until SHEET_URL is set and refresh_all is called)
tasks_df = pd.DataFrame(columns=TASKS_HEADER)
logs_df = pd.DataFrame(columns=LOGS_HEADER)
clips_df = pd.DataFrame(columns=CLIPS_HEADER)

print("\nPlease manually set the SHEET_URL variable in this cell and run it again.")
print("After setting the URL, run the next cell to initialize worksheets and load initial data.")

Successfully connected to spreadsheet: ProductivityApp
Worksheets 'tasks', 'pomodoro_logs', and 'web_clips' are ready.

Please manually set the SHEET_URL variable in this cell and run it again.
After setting the URL, run the next cell to initialize worksheets and load initial data.


In [113]:
# AI Plan (Gemini or Rule-based)
import google.generativeai as genai
from google.colab import userdata

def generate_today_plan():
    global tasks_df
    # Access API key and configure model inside the function
    api_key = None
    model = None
    try:
        api_key = userdata.get('GOOGLE_API_KEY')
        if api_key:
            genai.configure(api_key=api_key)
            try:
                model = genai.GenerativeModel('gemini-2.5-flash')
            except Exception as e:
                print(f"Warning: Gemini model 'gemini-2.5-flash' not available. AI Plan will use rule-based method. Error: {e}")
                model = None # Fallback to rule-based
        else:
            print("Warning: GOOGLE_API_KEY not set. AI Plan will use rule-based method.")
    except Exception as e:
         print(f"Warning: Could not access GOOGLE_API_KEY Secret. AI Plan will use rule-based method. Error: {e}")
         model = None # Fallback to rule-based


    today = tznow().date().isoformat()
    cand = tasks_df[
        ((tasks_df["due_date"]==today) | (tasks_df["planned_for"].str.lower()=="today")) &
        (tasks_df["status"]!="done")
    ].copy()
    if cand.empty:
        return "今天沒有標記的任務. 請在 Tasks 分頁把任務的 due_date 設為今天或 planned_for 設為 today."

    pr_order = {"H":0, "M":1, "L":2}
    cand["p_ord"] = cand["priority"].map(pr_order).fillna(3)
    cand = cand.sort_values(["p_ord","est_min"], ascending=[True, True])

    plan_md = ""
    if model: # Use Gemini if available
        print("Using AI model for planning.")
        sys_prompt = (
            "你是一位任務規劃助理. 請把輸入的任務 (含估時與優先級) 排成三段: morning, afternoon, evening,"
            "並給出每段的重點, 順序, 每項的時間預估與備註. 總時數請大致符合任務估時總和."
            "回傳以 Markdown 條列, 格式:\n"
            "### Morning\n- [任務ID] 任務名稱 (預估 xx 分) - 備註\n...\n"
            "### Afternoon\n...\n### Evening\n...\n"
        )
        items = []
        for _, r in cand.iterrows():
            items.append({
                "id": r["id"], "task": r["task"], "est_min": int(r["est_min"]),
                "priority": r["priority"]
            })
        user_content = json.dumps({"today": today, "tasks": items}, ensure_ascii=False)
        try:
            # Use a more robust safety setting if needed
            resp = model.generate_content(sys_prompt + "\n\n" + user_content)
            plan_md = resp.text
        except Exception as e:
            plan_md = f"AI 規劃失敗: {e}\n\n改用規則式規劃."
            print("AI planning failed, falling back to rule-based.")
    else:
        plan_md = "未設定 Google API Key 或模型不可用, 使用規則式規劃.\n\n"
        print("Using rule-based planning.")


    # Rule-based fallback or addition
    buckets = {"morning": [], "afternoon": [], "evening": []}
    total = len(cand)
    for i, (_, r) in enumerate(cand.iterrows()):
        if i % 3 == 0:
            buckets["morning"].append(r)
        elif i % 3 == 1:
            buckets["afternoon"].append(r)
        else:
            buckets["evening"].append(r)

    def sec_md(name, rows):
        if not rows: return f"### {name.title()}\n(無)\n"
        lines = [f"### {name.title()}"]
        for r in rows:
            lines.append(f"- [{r['id']}] {r['task']} (預估 {int(r['est_min'])} 分, P:{r['priority']})")
        return "\n".join(lines) + "\n"

    rule_md = sec_md("morning", buckets["morning"]) + "\n" + \
              sec_md("afternoon", buckets["afternoon"]) + "\n" + \
              sec_md("evening", buckets["evening"])

    # Combine AI plan and rule-based plan for comparison or fallback
    if plan_md and "AI 規劃失敗" not in plan_md and "未設定 Google API Key" not in plan_md: # If AI plan was successful
         return plan_md + "\n---\n**規則式規劃 (僅供參考):**\n" + rule_md.strip()
    else: # If AI plan failed or not attempted
         return plan_md.strip() + "\n" + rule_md.strip()

In [114]:
# Gradio Interface
def _refresh_ui():
    """Refreshes data from sheets and updates UI components."""
    global tasks_df, logs_df, clips_df
    tasks_df, logs_df, clips_df = refresh_all()
    # Return updated components
    return (
        gr.Dataframe(value=tasks_df), # Correct way to return updated Dataframe
        gr.Dataframe(value=logs_df),
        gr.Dataframe(value=clips_df),
        gr.Dropdown(choices=list_task_choices()), # Correct way to return updated Dropdown
        gr.Dropdown(choices=list_task_choices()), # For sel_task_dropdown
        gr.Dropdown(choices=["all"] + tasks_df["status"].unique().tolist()), # For query_status_dropdown
        gr.Markdown(value=today_summary())
    )

# Helper function to update task choices dropdown after task changes
def _update_task_choices():
    # This function is no longer needed as _refresh_ui handles dropdown updates
    return gr.Dropdown(choices=list_task_choices()) # Correct way to return updated Dropdown


with gr.Blocks(title="待辦清單與番茄鐘應用") as demo:
    gr.Markdown("# 待辦清單與番茄鐘應用")
    with gr.Row():
        btn_refresh = gr.Button("重新整理 (Sheet -> App)")
        out_summary = gr.Markdown(today_summary()) # Initial summary

    with gr.Tab("任務管理"):
        gr.Markdown("### 新增任務")
        with gr.Row():
            with gr.Column(scale=2):
                task_input = gr.Textbox(label="任務名稱", placeholder="例如：寫報告")
                priority_input = gr.Dropdown(["H","M","L"], value="M", label="優先級")
                est_min_input = gr.Number(value=25, label="預估時間 (分鐘)", precision=0)
                due_date_input = gr.Textbox(label="到期日 (YYYY-MM-DD, 可選)", placeholder="例如：2023-12-31")
                labels_input = gr.Textbox(label="標籤 (逗號分隔, 可選)", placeholder="例如：工作, 緊急")
                notes_input = gr.Textbox(label="備註 (可選)")
                planned_for_input = gr.Dropdown(["","today","tomorrow"], value="", label="計畫歸屬")
                btn_add = gr.Button("新增任務")
                msg_add = gr.Markdown()
            with gr.Column(scale=3):
                grid_tasks = gr.Dataframe(value=tasks_df, label="任務清單 (來自 Google Sheet)", interactive=False)

        gr.Markdown("### 更新或刪除任務")
        with gr.Row():
            # Using the task_choice dropdown defined later for consistency
            task_choice_dropdown = gr.Dropdown(choices=list_task_choices(), label="選取任務")
            new_status_dropdown = gr.Dropdown(["todo","in-progress","done"], value="in-progress", label="更新狀態為")
            btn_update = gr.Button("更新狀態")
            btn_done = gr.Button("直接標記為完成")
            btn_delete = gr.Button("刪除任務")
            msg_update_delete = gr.Markdown()

        gr.Markdown("### 任務查詢")
        with gr.Row():
            query_status_dropdown = gr.Dropdown(choices=["all"] + tasks_df["status"].unique().tolist(), label="狀態", value="all") # Initial choices
            date_type_radio = gr.Radio(["created_at", "due_date", "completed_at"], label="依據日期類型", value="created_at")
            start_date_input = gr.Textbox(label="開始日期 (YYYY-MM-DD, 可選)", placeholder="例如：2023-01-01")
            end_date_input = gr.Textbox(label="結束日期 (YYYY-MM-DD, 可選)", placeholder="例如：2023-12-31")
            btn_query = gr.Button("查詢任務")
        grid_query_results = gr.Dataframe(label="查詢結果", interactive=False)


    with gr.Tab("番茄鐘"):
        gr.Markdown("### 番茄鐘計時")
        with gr.Row():
            sel_task_dropdown = gr.Dropdown(choices=list_task_choices(), label="選擇任務")
            cycles_input = gr.Number(value=1, precision=0, label="番茄數 (僅作紀錄)")
        with gr.Row():
            btn_start_work = gr.Button("開始工作階段")
            note_work_input = gr.Textbox(label="工作備註 (可選)")
            btn_end_work = gr.Button("結束工作階段並記錄")
        with gr.Row():
            btn_start_break = gr.Button("開始休息階段")
            note_break_input = gr.Textbox(label="休息備註 (可選)")
            btn_end_break = gr.Button("結束休息階段並記錄")
        msg_pomo = gr.Markdown()
        grid_logs = gr.Dataframe(value=logs_df, label="番茄鐘紀錄", interactive=False)

    with gr.Tab("AI 計畫"):
        gr.Markdown("根據設定為**今天**的任務（`due_date` 或 `planned_for` 包含今天），產生 **morning / afternoon / evening** 三段行動計畫。若未設定 Google API Key 或模型不可用，會使用規則式規劃。")
        btn_plan = gr.Button("產生今日計畫")
        out_plan = gr.Markdown()

    with gr.Tab("網頁擷取 (Crawler)"):
        gr.Markdown("從指定網頁擷取內容並轉換為任務。")
        with gr.Row():
            url_input = gr.Textbox(label="目標 URL", placeholder="例如：https://example.com")
            selector_input = gr.Textbox(label="CSS Selector", placeholder="例如：a.news-item")
            mode_radio = gr.Radio(["text","href","both"], value="text", label="擷取內容")
            limit_number = gr.Number(value=20, precision=0, label="最多擷取幾筆")
            btn_crawl = gr.Button("開始擷取")
            msg_crawl = gr.Markdown()
        grid_clips = gr.Dataframe(value=clips_df, label="擷取結果 (會同步寫入 Sheet)", interactive=True)
        gr.Markdown("### 將擷取結果加入為任務")
        with gr.Row():
            clip_ids_input = gr.Textbox(label="要加入任務的 clip_id (多個以逗號分隔)", placeholder="例如：abc123, def456")
            clip_priority_dropdown = gr.Dropdown(["H","M","L"], value="L", label="新增任務優先級")
            clip_est_number = gr.Number(value=25, precision=0, label="新增任務預估分鐘")
            btn_add_clips = gr.Button("將勾選的擷取項目加入為任務")
            msg_add_clips = gr.Markdown()


    with gr.Tab("匯出與匯入"):
        with gr.Row():
            with gr.Column():
                gr.Markdown("### 匯出資料")
                export_format_radio = gr.Radio(["csv", "json"], value="csv", label="匯出格式")
                btn_export_tasks = gr.Button("匯出任務資料")
                out_export_tasks = gr.Textbox(label="任務資料 (CSV/JSON)")
                btn_export_logs = gr.Button("匯出番茄鐘紀錄資料")
                out_export_logs = gr.Textbox(label="番茄鐘紀錄資料 (CSV/JSON)")
            with gr.Column():
                gr.Markdown("### 匯入資料 (注意: 匯入將覆蓋現有資料!)")
                import_format_radio = gr.Radio(["csv", "json"], value="csv", label="匯入格式")
                in_import_tasks = gr.Textbox(label="貼上任務資料 (CSV/JSON)")
                btn_import_tasks = gr.Button("匯入任務資料")
                msg_import_tasks = gr.Markdown()
                in_import_logs = gr.Textbox(label="貼上番茄鐘紀錄資料 (CSV/JSON)")
                btn_import_logs = gr.Button("匯入番茄鐘紀錄資料")
                msg_import_logs = gr.Markdown()

    with gr.Tab("統計"):
        gr.Markdown("### 視覺化統計圖表")
        with gr.Row():
            with gr.Column():
                gr.Markdown("#### 每日任務完成數")
                btn_analyze_completion = gr.Button("分析並顯示每日任務完成數")
                plot_completion_output = gr.Plot()
            with gr.Column():
                gr.Markdown("#### 番茄鐘時長分佈")
                btn_analyze_pomodoro = gr.Button("分析並顯示番茄鐘時長分佈")
                plot_pomodoro_output = gr.Plot()


    # === Bind Actions ===
    # Refresh button updates all main components
    btn_refresh.click(
        _refresh_ui,
        outputs=[grid_tasks, grid_logs, grid_clips, task_choice_dropdown, sel_task_dropdown, query_status_dropdown, out_summary]
    )

    # Initial UI update on load (optional, but good for showing data immediately)
    demo.load(
        _refresh_ui,
        outputs=[grid_tasks, grid_logs, grid_clips, task_choice_dropdown, sel_task_dropdown, query_status_dropdown, out_summary]
    )


    # Add task
    btn_add.click(
        add_task,
        inputs=[task_input, priority_input, est_min_input, due_date_input, labels_input, notes_input, planned_for_input],
        outputs=[msg_add, grid_tasks]
    ).success(
        _refresh_ui, # Refresh all UI components after adding
        outputs=[grid_tasks, grid_logs, grid_clips, task_choice_dropdown, sel_task_dropdown, query_status_dropdown, out_summary]
    )


    # Update task status
    btn_update.click(
        update_task_status,
        inputs=[task_choice_dropdown, new_status_dropdown],
        outputs=[msg_update_delete, grid_tasks]
    ).success(
        _refresh_ui, # Refresh all UI components after updating
         outputs=[grid_tasks, grid_logs, grid_clips, task_choice_dropdown, sel_task_dropdown, query_status_dropdown, out_summary]
    )

    # Mark task done
    btn_done.click(
        mark_done,
        inputs=[task_choice_dropdown],
        outputs=[msg_update_delete, grid_tasks]
    ).success(
        _refresh_ui, # Refresh all UI components after marking done
        outputs=[grid_tasks, grid_logs, grid_clips, task_choice_dropdown, sel_task_dropdown, query_status_dropdown, out_summary]
    )

    # Delete task
    btn_delete.click(
        delete_task,
        inputs=[task_choice_dropdown],
        outputs=[msg_update_delete, grid_tasks]
    ).success(
        _refresh_ui, # Refresh all UI components after deleting
        outputs=[grid_tasks, grid_logs, grid_clips, task_choice_dropdown, sel_task_dropdown, query_status_dropdown, out_summary]
    )


    # Pomodoro start/end
    btn_start_work.click(
        start_phase, inputs=[sel_task_dropdown, gr.State("work"), cycles_input], outputs=[msg_pomo]
    )
    btn_end_work.click(
        end_phase, inputs=[sel_task_dropdown, note_work_input], outputs=[msg_pomo]
    ).success(
        _refresh_ui, # Refresh all UI components after ending phase
        outputs=[grid_tasks, grid_logs, grid_clips, task_choice_dropdown, sel_task_dropdown, query_status_dropdown, out_summary]
    )
    btn_start_break.click(
        start_phase, inputs=[sel_task_dropdown, gr.State("break"), cycles_input], outputs=[msg_pomo]
    )
    btn_end_break.click(
        end_phase, inputs=[sel_task_dropdown, note_break_input], outputs=[msg_pomo]
    ).success(
        _refresh_ui, # Refresh all UI components after ending phase
        outputs=[grid_tasks, grid_logs, grid_clips, task_choice_dropdown, sel_task_dropdown, query_status_dropdown, out_summary]
    )


    # AI Plan
    btn_plan.click(generate_today_plan, outputs=[out_plan])

    # Crawler
    def _crawl_and_save(u, s, m, l):
        df, msg = crawl(u, s, m, l)
        global clips_df
        if not df.empty:
            # Avoid adding duplicate clips if crawling same URL/selector multiple times
            # Simple approach: check if url and text/href combination exists
            existing_clips = set(zip(clips_df['url'], clips_df['text'], clips_df['href']))
            new_rows = [row for _, row in df.iterrows() if (row['url'], row['text'], row['href']) not in existing_clips]

            if new_rows:
                clips_df = pd.concat([clips_df, pd.DataFrame(new_rows)], ignore_index=True)
                write_df(ws_clips, clips_df, CLIPS_HEADER)
                msg += f", 新增 {len(new_rows)} 筆到 Sheet"
            else:
                msg += ", 沒有新項目加入 Sheet"

        return msg, clips_df

    btn_crawl.click(_crawl_and_save, inputs=[url_input, selector_input, mode_radio, limit_number], outputs=[msg_crawl, grid_clips])

    def _add_clips_action(clip_ids_str, pr, est):
        ids = [c.strip() for c in (clip_ids_str or "").split(",") if c.strip()]
        msg, new_clips, new_tasks = add_clips_as_tasks(ids, pr, est)
        # After adding clips as tasks, refresh the tasks grid and dropdowns
        # tasks_df, logs_df, clips_df = refresh_all() # Ensure all dataframes are updated - _refresh_ui does this
        return msg, new_clips, new_tasks, list_task_choices(), list_task_choices()

    btn_add_clips.click(
        _add_clips_action,
        inputs=[clip_ids_input, clip_priority_dropdown, clip_est_number],
        outputs=[msg_add_clips, grid_clips, grid_tasks, task_choice_dropdown, sel_task_dropdown]
    ).success(
         _refresh_ui, # Refresh all UI components after adding clips as tasks
         outputs=[grid_tasks, grid_logs, grid_clips, task_choice_dropdown, sel_task_dropdown, query_status_dropdown, out_summary]
    )

    # Export/Import bindings
    btn_export_tasks.click(lambda fmt: export_data(tasks_df, fmt), inputs=[export_format_radio], outputs=[out_export_tasks])
    btn_export_logs.click(lambda fmt: export_data(logs_df, fmt), inputs=[export_format_radio], outputs=[out_export_logs])

    def _import_tasks_action(data_string, format):
        global tasks_df
        imported_df, msg = import_data(data_string, format, TASKS_HEADER)
        if msg.startswith("資料匯入成功"):
            tasks_df = imported_df # Simple replace for now
            write_df(ws_tasks, tasks_df, TASKS_HEADER)
            # Also update task choices dropdowns
            return msg, tasks_df, list_task_choices(), list_task_choices()
        return msg, tasks_df, list_task_choices(), list_task_choices() # Return current state on failure

    btn_import_tasks.click(
        _import_tasks_action,
        inputs=[in_import_tasks, import_format_radio],
        outputs=[msg_import_tasks, grid_tasks, task_choice_dropdown, sel_task_dropdown]
    ).success(
         _refresh_ui, # Refresh all UI components after importing tasks
         outputs=[grid_tasks, grid_logs, grid_clips, task_choice_dropdown, sel_task_dropdown, query_status_dropdown, out_summary]
    )

    def _import_logs_action(data_string, format):
        global logs_df
        imported_df, msg = import_data(data_string, format, LOGS_HEADER)
        if msg.startswith("資料匯入成功"):
            logs_df = imported_df # Simple replace for now
            write_df(ws_logs, logs_df, LOGS_HEADER)
            return msg, logs_df
        return msg, logs_df # Return current state on failure

    btn_import_logs.click(
        _import_logs_action,
        inputs=[in_import_logs, import_format_radio],
        outputs=[msg_import_logs, grid_logs]
    ).success(
         _refresh_ui, # Refresh all UI components after importing logs
         outputs=[grid_tasks, grid_logs, grid_clips, task_choice_dropdown, sel_task_dropdown, query_status_dropdown, out_summary]
    )

    # Bind query function
    btn_query.click(
        query_tasks,
        inputs=[query_status_dropdown, start_date_input, end_date_input, date_type_radio],
        outputs=[grid_query_results]
    )

    # Bind statistics functions
    btn_analyze_completion.click(
        lambda: create_completion_plot(analyze_task_completion_daily(tasks_df)),
        outputs=[plot_completion_output]
    )
    btn_analyze_pomodoro.click(
        lambda: create_pomodoro_plot(analyze_pomodoro_distribution(logs_df)),
        outputs=[plot_pomodoro_output]
    )


demo.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://53ea6da1e1d99492ca.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/tmp/ipython-input-2185084888.py:39: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7867 <> https://53ea6da1e1d99492ca.gradio.live


In [115]:
# Crawler Functions
def crawl(url, selector, mode, limit):
    try:
        resp = requests.get(url, timeout=15, headers={"User-Agent":"Mozilla/5.0"})
        resp.raise_for_status()
    except Exception as e:
        return pd.DataFrame(columns=CLIPS_HEADER), f"請求失敗: {e}"

    soup = BeautifulSoup(resp.text, "html.parser")
    nodes = soup.select(selector)
    rows = []
    for i, n in enumerate(nodes[:int(limit) if limit else 20]):
        text = n.get_text(strip=True) if mode in ("text","both") else ""
        href = n.get("href") if mode in ("href","both") else ""
        if href and href.startswith("/"):
            href = urljoin(url, href)
        rows.append({
            "clip_id": str(uuid.uuid4())[:8],
            "url": url,
            "selector": selector,
            "text": text,
            "href": href,
            "created_at": tznow().isoformat(),
            "added_to_task": ""
        })
    df = pd.DataFrame(rows, columns=CLIPS_HEADER)
    return df, f"擷取 {len(df)} 筆"

def add_clips_as_tasks(clip_ids, default_priority, est_min):
    global clips_df, tasks_df
    if not clip_ids:
        return "請先勾選要加入的爬蟲項目", clips_df, tasks_df
    sel = clips_df[clips_df["clip_id"].isin(clip_ids)]
    _now = tznow().isoformat()
    new_tasks = []
    for _, r in sel.iterrows():
        title = r["text"] or r["href"] or "(未命名)"
        note = f"來源: {r['url']}\n選擇器: {r['selector']}\n連結: {r['href']}"
        new_tasks.append({
            "id": str(uuid.uuid4())[:8],
            "task": title[:120], # Truncate long titles
            "status": "todo",
            "priority": default_priority or "M",
            "est_min": int(est_min) if est_min else 25,
            "start_time": "",
            "end_time": "",
            "actual_min": 0,
            "pomodoros": 0,
            "due_date": "",
            "labels": "from:crawler",
            "notes": note,
            "created_at": _now,
            "updated_at": _now,
            "completed_at": "",
            "planned_for": ""
        })
    if new_tasks:
        tasks_df = pd.concat([tasks_df, pd.DataFrame(new_tasks)], ignore_index=True)
        clips_df.loc[clips_df["clip_id"].isin(clip_ids), "added_to_task"] = "yes"
        write_df(ws_tasks, tasks_df, TASKS_HEADER)
        write_df(ws_clips, clips_df, CLIPS_HEADER)
        return f"已加入 {len(new_tasks)} 項為任務", clips_df, tasks_df
    return "無可加入項目", clips_df, tasks_df

In [116]:
# Statistics and Visualization (Summary)
def today_summary():
    global tasks_df
    today = tznow().date().isoformat()
    planned = tasks_df[
        ((tasks_df["due_date"]==today) | (tasks_df["planned_for"].str.lower()=="today"))
    ]
    done = planned[planned["status"]=="done"]
    total = len(planned)
    done_n = len(done)
    rate = (done_n/total*100) if total>0 else 0
    return f"今日計畫任務: {total}; 完成: {done_n}; 完成率: {rate:.1f}%"

In [117]:
# Export/Import Functions
def export_data(df, format):
    if df.empty:
        return ""
    if format == 'csv':
        return df.to_csv(index=False)
    elif format == 'json':
        return df.to_json(indent=2, orient='records')
    else:
        return ""

def import_data(data_string, format, header):
    if not data_string:
        return pd.DataFrame(columns=header), "無資料可匯入"

    try:
        if format == 'csv':
            df = pd.read_csv(io.StringIO(data_string))
        elif format == 'json':
            df = pd.read_json(io.StringIO(data_string))
        else:
            return pd.DataFrame(columns=header), "不支援的格式"

        if df.empty:
            return pd.DataFrame(columns=header), "匯入資料為空"

        # Ensure columns match the header and reorder
        for col in header:
            if col not in df.columns:
                df[col] = ""
        df = df[header]

        # Apply type conversions similar to read_df
        if "est_min" in df.columns:
            df["est_min"] = pd.to_numeric(df["est_min"], errors="coerce").fillna(0).astype(int)
        if "actual_min" in df.columns:
            df["actual_min"] = pd.to_numeric(df["actual_min"], errors="coerce").fillna(0).astype(int)
        if "pomodoros" in df.columns:
            df["pomodoros"] = pd.to_numeric(df["pomodoros"], errors="coerce").fillna(0).astype(int)

        return df, "資料匯入成功"

    except Exception as e:
        return pd.DataFrame(columns=header), f"匯入失敗: {e}"

In [118]:
# Pomodoro Functions
_active_sessions = {}

def start_phase(task_id, phase, cycles):
    if not task_id:
        return "請先選擇任務"
    if task_id in _active_sessions:
         return "該任務已有進行中的階段" # Prevent multiple sessions for the same task
    _active_sessions[task_id] = {
        "phase": phase,
        "start_ts": tznow().isoformat(),
        "cycles": int(cycles) if cycles else 1
    }
    return f"已開始: {phase} (task: {task_id})"

def end_phase(task_id, note):
    global logs_df, tasks_df
    if task_id not in _active_sessions:
        return "尚未開始任何階段"
    sess = _active_sessions.pop(task_id)
    start = pd.to_datetime(sess["start_ts"])
    end = tznow()
    minutes = round((end - start).total_seconds() / 60.0, 2)

    # Add a check for minimum session duration if needed (e.g., > 1 minute)
    if minutes < 0.1: # Example: less than 6 seconds
         return f"階段過短 ({minutes:.2f} 分鐘), 未記錄."

    log = pd.DataFrame([{
        "log_id": str(uuid.uuid4())[:8],
        "task_id": task_id,
        "phase": sess["phase"],
        "start_ts": start.isoformat(),
        "end_ts": end.isoformat(),
        "minutes": minutes,
        "cycles": int(sess["cycles"]),
        "note": note or ""
    }])
    logs_df = pd.concat([logs_df, log], ignore_index=True)
    write_df(ws_logs, logs_df, LOGS_HEADER)

    if sess["phase"] == "work":
        recalc_task_actuals(task_id)
        write_df(ws_tasks, tasks_df, TASKS_HEADER)

    return f"已結束: {sess['phase']}, 記錄 {minutes} 分鐘"

def recalc_task_actuals(task_id):
    """根據 logs_df 回寫 actual_min 與 pomodoros"""
    global tasks_df, logs_df
    work_logs = logs_df[(logs_df["task_id"]==task_id) & (logs_df["phase"]=="work")]
    total_min = work_logs["minutes"].astype(float).sum() if not work_logs.empty else 0
    pomos = int(round(total_min / 25.0)) # Assuming 25 min pomodoro
    idx = tasks_df.index[tasks_df["id"]==task_id]
    if len(idx)==0: return
    i = idx[0]
    tasks_df.loc[i,"actual_min"] = int(total_min)
    tasks_df.loc[i,"pomodoros"] = pomos
    tasks_df.loc[i,"updated_at"] = tznow().isoformat()

In [119]:
# Task Management Functions
def add_task(task, priority, est_min, due_date, labels, notes, planned_for):
    global tasks_df, ws_tasks
    if ws_tasks is None:
        return "錯誤：Google Sheet 工作表未初始化，請先執行設定區塊", tasks_df

    _now = tznow().isoformat()
    new = pd.DataFrame([{
        "id": str(uuid.uuid4())[:8],
        "task": task.strip(),
        "status": "todo",
        "priority": priority or "M",
        "est_min": int(est_min) if est_min else 25,
        "start_time": "",
        "end_time": "",
        "actual_min": 0,
        "pomodoros": 0,
        "due_date": due_date or "",
        "labels": labels or "",
        "notes": notes or "",
        "created_at": _now,
        "updated_at": _now,
        "completed_at": "",
        "planned_for": planned_for or ""
    }])
    tasks_df = pd.concat([tasks_df, new], ignore_index=True)
    write_df(ws_tasks, tasks_df, TASKS_HEADER)
    return "任務新增成功", tasks_df

def update_task_status(task_id, new_status):
    global tasks_df, ws_tasks
    if ws_tasks is None:
        return "錯誤：Google Sheet 工作表未初始化，請先執行設定區塊", tasks_df

    idx = tasks_df.index[tasks_df["id"] == task_id]
    if len(idx)==0:
        return "找不到指定任務", tasks_df
    i = idx[0]
    tasks_df.loc[i, "status"] = new_status
    tasks_df.loc[i, "updated_at"] = tznow().isoformat()
    if new_status == "done" and not tasks_df.loc[i, "completed_at"]:
        tasks_df.loc[i, "completed_at"] = tznow().isoformat()
    write_df(ws_tasks, tasks_df, TASKS_HEADER)
    return "任務狀態已更新", tasks_df

def mark_done(task_id):
    return update_task_status(task_id, "done")

def delete_task(task_id):
    global tasks_df, ws_tasks
    if ws_tasks is None:
        return "錯誤：Google Sheet 工作表未初始化，請先執行設定區塊", tasks_df

    initial_rows = len(tasks_df)
    tasks_df = tasks_df[tasks_df["id"] != task_id].reset_index(drop=True)

    if len(tasks_df) < initial_rows:
        write_df(ws_tasks, tasks_df, TASKS_HEADER)
        return f"任務 (ID: {task_id}) 已刪除", tasks_df
    else:
        return "找不到指定任務", tasks_df


def list_task_choices():
    global tasks_df
    if tasks_df.empty:
        return []
    # Sort by priority and then by task name for better readability
    pr_order = {"H":0, "M":1, "L":2}
    sorted_tasks = tasks_df.copy()
    sorted_tasks["_p_ord"] = sorted_tasks["priority"].map(pr_order).fillna(3)
    sorted_tasks = sorted_tasks.sort_values(["_p_ord", "task"]).drop(columns=["_p_ord"])

    def row_label(r):
        return f"[{r['status']}] (優先級:{r['priority']}) {r['task']} - ID:{r['id']}"

    # Return value and display label
    return [(row_label(r), r["id"]) for _, r in sorted_tasks.iterrows()]

# Helper function to refresh data and update UI components
def _refresh_data_and_ui():
    global tasks_df, logs_df, clips_df
    tasks_df, logs_df, clips_df = refresh_all()
    return tasks_df, logs_df, clips_df, list_task_choices(), today_summary()

截止76